# Sales Performance Analytics — Day 2
## KPI Calculation & Visualization
**Internship Task | Data Science**

**Objective:** Calculate all 6 business development KPIs from the cleaned dataset and visualize them using Python charts. Outputs will be used to build the Power BI dashboard.

### KPIs to Calculate:
1. Lead Conversion Rate
2. Customer Acquisition Cost (CAC)
3. Revenue per Client
4. Monthly Growth Rate
5. Sales Cycle Duration
6. Lead Source Performance

In [1]:
# ─────────────────────────────────────────
# Import Libraries
# ─────────────────────────────────────────

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Plot styling
plt.rcParams.update({
    'figure.dpi'      : 150,
    'axes.spines.top' : False,
    'axes.spines.right': False,
    'font.family'     : 'DejaVu Sans',
    'axes.titlesize'  : 13,
    'axes.titleweight': 'bold'
})

sns.set_palette("Set2")

pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# ─────────────────────────────────────────
# Load Cleaned Dataset
# ─────────────────────────────────────────

df = pd.read_csv('../data/cleaned/BD_KPI_Cleaned_Dataset.csv',
                 parse_dates=['Lead_Date', 'Conversion_Date'])

# Restore period column
df['Lead_Month'] = df['Lead_Date'].dt.to_period('M')

print(f"Cleaned dataset loaded")
print(f"   Shape  : {df.shape}")
print(f"   Columns: {df.columns}")
print(f"\nFirst 10 rows:")
df.head(10)

Cleaned dataset loaded
   Shape  : (683, 13)
   Columns: Index(['Lead_ID', 'Lead_Source', 'Lead_Date', 'Conversion_Status',
       'Conversion_Date', 'Revenue_Value', 'Marketing_Cost',
       'Sales_Representative', 'Region', 'Sales_Cycle_Days', 'Lead_Month',
       'Lead_Year', 'Is_Converted'],
      dtype='object')

First 10 rows:


,Lead_ID,Lead_Source,Lead_Date,Conversion_Status,Conversion_Date,Revenue_Value,Marketing_Cost,Sales_Representative,Region,Sales_Cycle_Days,Lead_Month,Lead_Year,Is_Converted
0,LEAD-0043,Referral,2024-04-02,Not Converted,NaT,0.00,3036.10,David Brown,East,NaN,2024-04,2024,0
1,LEAD-0199,Referral,2024-02-07,Converted,2024-06-06,10390.10,2639.44,Unassigned,West,120.00,2024-02,2024,1
2,LEAD-0526,Email Campaign,2023-02-11,Not Converted,NaT,0.00,3021.93,David Brown,West,NaN,2023-02,2023,0
3,LEAD-0638,Email Campaign,2023-09-09,Not Converted,NaT,0.00,2639.44,David Brown,South,NaN,2023-09,2023,0
4,LEAD-0067,Website,2023-04-29,Not Converted,NaT,0.00,2095.58,Unassigned,East,NaN,2023-04,2023,0
5,LEAD-0130,Email Campaign,2023-10-01,Not Converted,NaT,0.00,4120.59,Alice Johnson,North,NaN,2023-10,2023,0
6,LEAD-0567,Social Media,2024-06-22,Converted,2024-08-15,12684.79,1676.86,Alice Johnson,South,54.00,2024-06,2024,1
7,LEAD-0626,Referral,2024-03-09,Not Converted,NaT,0.00,3148.67,Unassigned,North,NaN,2024-03,2024,0
8,LEAD-0680,Direct,2024-03-22,Not Converted,NaT,0.00,3497.45,Unassigned,West,NaN,2024-03,2024,0
9,LEAD-0155,Direct,2023-09-28,Converted,2023-10-23,14310.82,4912.19,Unassigned,East,25.00,2023-09,2023,1
